In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

# Imports for the openwifi self cap files
from csi_processing_tools.openwifi.plot_csi import plot, plot_single, read_from_file

# Imports for the ath9k cap files
from csi_processing_tools.ath9k import plotter, reader

In [2]:
def normalize(csi_packets, debug=False):
    normalized = list()
    for csi_matrix in csi_packets:
        norm = (csi_matrix[2:-2]) / np.sqrt((np.sum(np.abs(csi_matrix[2:-2])**2)))
        if debug == True:
            print(np.min(norm))
        if not np.isnan(np.min(norm)):
            normalized.append(norm)
    return normalized

In [ ]:
## Generating a learning data set
base_path = "csi_captures/training/"

# Reading the openwifi selfcap files
openwifi_path = base_path + "/openwifi/"
fuzzed_openwifi = list()
fuzzed_openwifi.append(normalize(read_from_file(openwifi_path + "fuzzed_0_-20_0_20.txt",filter_value=300)))
fuzzed_openwifi.append(normalize(read_from_file(openwifi_path + "fuzzed_0_7_1_-33.txt",filter_value=300)))
unfuzzed_openwifi = normalize(read_from_file(openwifi_path + "unfuzzed.txt",filter_value=300))

# Reading the ath9k cap files
ath9k_path = base_path + "/ath9k/"
fuzzed_ath9k = list()
fuzzed_ath9k.append(normalize(reader.read_csi_file(ath9k_path + "0_-20_0_-20_fuzzed.dat")[20:]))
fuzzed_ath9k.append(normalize(reader.read_csi_file(ath9k_path + "1_-15_0_23_fuzzed.dat")[20:]))
unfuzzed_ath9k = normalize(reader.read_csi_file(ath9k_path + "unfuzzed.dat"))


In [7]:
## Generate a validation data set
base_path = "csi_captures/validation"

# Reading the openwifi selfcap files
openwifi_path = base_path + "/openwifi/"
fuzzed_openwifi_validation = list()
fuzzed_openwifi_validation.append(normalize(read_from_file(openwifi_path + "openwifi_0_-20_0_20.txt",filter_value=300)))
fuzzed_openwifi_validation.append(normalize(read_from_file(openwifi_path + "openwifi_0_7_1_-33.txt",filter_value=300)))
unfuzzed_openwifi_validation = normalize(read_from_file(openwifi_path + "openwifi_unfuzzed.txt",filter_value=300))

# Reading the ath9k cap files
ath9k_path = base_path + "/ath9k/"
fuzzed_ath9k_validation = list()
fuzzed_ath9k_validation.append(normalize(reader.read_csi_file(ath9k_path + "fuzzed_0_-20_0_-20.dat")))
fuzzed_ath9k_validation.append(normalize(reader.read_csi_file(ath9k_path + "fuzzed_1_-15_0_23.dat")))
unfuzzed_ath9k_validation = normalize(reader.read_csi_file(ath9k_path + "unfuzzed.dat"))


In [4]:
def NMSE(x, y):
    numerator = np.sum(np.abs(y-x))
    denominator = np.sum(np.abs(y))
    return 10*np.log10(numerator/denominator)

In [36]:
# Loop to check how defuzzing depends on the amount of packets averaged

openwifi_avg_nmse = [list() for _ in range(len(fuzzed_openwifi))]
openwifi_snr= list()
ath9k_avg_nmse = [list() for _ in range(len(fuzzed_ath9k))]
ath9k_snr = list()

for amount in range(1,50):

    # Average certain amount of packages to be used for divsion
    openwifi_unfuzzed_total = np.zeros(52, dtype='complex128')
    openwifi_fuzzed_total = np.array([np.zeros(52, dtype='complex128')] * len(fuzzed_openwifi))
    openwifi_unfuzzed_validation_total = np.zeros(52, dtype='complex128')
    
    ath9k_unfuzzed_total = np.zeros(52, dtype='complex128')
    ath9k_fuzzed_total = np.array([np.zeros(52, dtype='complex128')] * len(fuzzed_ath9k))
    ath9k_unfuzzed_validation_total = np.zeros(52, dtype='complex128')
    # Sum up # of packets
    for index in range(0, amount):
        openwifi_unfuzzed_total += unfuzzed_openwifi[random.randint(0,len(unfuzzed_openwifi)-1)]
        openwifi_unfuzzed_validation_total += unfuzzed_openwifi_validation[random.randint(0,len(unfuzzed_openwifi_validation)-1)]
        # Add packet to current total for every openwifi CSI file
        for item in range(0,len(fuzzed_openwifi)):
            openwifi_fuzzed_total[item] += fuzzed_openwifi[item][random.randint(0,len(fuzzed_openwifi[item])-1)]
        
        ath9k_unfuzzed_total += unfuzzed_ath9k[random.randint(0,len(unfuzzed_ath9k)-1)]
        ath9k_unfuzzed_validation_total += unfuzzed_ath9k_validation[random.randint(0,len(unfuzzed_ath9k_validation)-1)]
        # Add packet to current total for every openwifi CSI file
        for item in range(0,len(fuzzed_ath9k)):
            ath9k_fuzzed_total[item] += fuzzed_ath9k[item][random.randint(0,len(fuzzed_ath9k[item])-1)]


    # Calculate the averages for the openwifi CSI
    openwifi_unfuzzed_avg = openwifi_unfuzzed_total/amount
    openwifi_unfuzzed_validation_avg = openwifi_unfuzzed_validation_total/amount
    openwifi_defuzzing_patterns = list()
    for total in openwifi_fuzzed_total:
        avg = np.array(total)/amount
        openwifi_defuzzing_patterns.append(avg/openwifi_unfuzzed_avg)
    

    # Calculate the averages for the ath9k CSI
    ath9k_unfuzzed_avg = np.array(ath9k_unfuzzed_total)/amount
    ath9k_unfuzzed_validation_avg = ath9k_unfuzzed_validation_total/amount
    ath9k_defuzzing_patterns = list()
    for total in ath9k_fuzzed_total:
        avg = np.array(total)/amount
        ath9k_defuzzing_patterns.append(avg/ath9k_unfuzzed_avg)



    n = 100

    # Calculate base line snr to compare defuzzing results
    
    openwifi_standard_error = list()
    temp = list(np.random.permutation(np.arange(0,min([len(x) for x in unfuzzed_openwifi_validation])))[:n])
    for index in temp:
        openwifi_standard_error.append(NMSE(unfuzzed_openwifi_validation[index],openwifi_unfuzzed_validation_avg))
    openwifi_snr.append(np.mean(openwifi_standard_error))


    ath9k_standard_error = list()
    temp = list(np.random.permutation(np.arange(0,min([len(x) for x in unfuzzed_ath9k_validation])))[:n])
    for index in temp:
        ath9k_standard_error.append(NMSE(unfuzzed_ath9k_validation[index],ath9k_unfuzzed_validation_avg))
    ath9k_snr.append(np.mean(ath9k_standard_error))


    # Chose some random fuzzed packets to unfuzz
    openwifi_indices = list(np.random.permutation(np.arange(0,min([len(x) for x in fuzzed_openwifi_validation])))[:n])
    ath9k_indices = list(np.random.permutation(np.arange(0,min([len(x) for x in fuzzed_ath9k])))[:n])


    # Defuzz the pattern and calculate the NMSE (openwifi CSI)
    # Save the average of 100 defuzzings
    for i,csi_matrix in enumerate(fuzzed_openwifi_validation):
        temp = list()
        for index in openwifi_indices:
            temp.append(NMSE(csi_matrix[index][2:-2]/openwifi_defuzzing_patterns[i][2:-2],openwifi_unfuzzed_validation_avg[2:-2]))
        openwifi_avg_nmse[i].append(np.mean(temp))


    # Defuzz the pattern and calculate the NMSE (ath9k CSI)
    for i,csi_matrix in enumerate(fuzzed_ath9k_validation):
        temp = list()
        for index in ath9k_indices:
            temp.append(NMSE(csi_matrix[index]/ath9k_defuzzing_patterns[i],ath9k_unfuzzed_validation_avg))
        ath9k_avg_nmse[i].append(np.mean(temp))



In [ ]:
defuzzed = fuzzed_openwifi_validation[0][1]/openwifi_defuzzing_patterns[0]
plt.plot(np.abs(defuzzed))
plt.plot(np.abs(openwifi_unfuzzed_validation_avg))
plt.xlabel("subcarrier idx")
plt.ylabel("normalised amplitude")
plt.legend(["defuzzed","validation"])
plt.show()



In [ ]:
for i in range(0,10):
    defuzzed = unfuzzed_ath9k_validation[i]
    plt.plot(defuzzed)
    plt.plot(np.abs(ath9k_unfuzzed_validation_avg))
    plt.xlabel("subcarrier idx")
    plt.ylabel("normalised amplitude")
    plt.legend(["defuzzed","validation"])
    plt.show()

In [11]:
openwifi_snr = np.array(openwifi_snr) - 0.5

In [20]:
def plot_nmse(nmse_array,legend, style='solid'):
    plt.plot(nmse_array, linestyle=style)
    plt.title("Error for # packets averaged")
    plt.ylabel("Error/avg amplitude (dB)")
    plt.xlabel("# packets averaged")
    plt.legend(legend)
    

In [18]:
unnormalised = list()
for index, elem in enumerate(openwifi_avg_nmse[0]):
    unnormalised.append(elem - openwifi_avg_nmse[1][index] - 12.5)

In [ ]:
# Plot the NMSE of the different openwifi CSI files
for file in openwifi_avg_nmse:
    plot_nmse(file,["openwifi_0_-20_0_20","openwifi_0_7_1_-33"])
plot_nmse(openwifi_snr,["openwifi_0_-20_0_20","openwifi_0_7_1_-33","baseline_error"])
plot_nmse(unnormalised,["openwifi_0_-20_0_20","openwifi_0_7_1_-33","baseline_error","unnormalised"],'dotted')

In [78]:
unnormalised2 = list()
for index, elem in enumerate(ath9k_avg_nmse[0]):
    unnormalised2.append( elem - ath9k_avg_nmse[1][index])

In [ ]:
# Plot the NMSE of the different Ath9k CSI files
for file in ath9k_avg_nmse:
    plot_nmse(file, ["ath9k_0_-20_0_-20","ath9k_1_-15_0_23"])
plot_nmse(ath9k_snr,["openwifi_0_-20_0_20","openwifi_0_7_1_-33","baseline_error"])
plot_nmse(unnormalised2,["openwifi_0_-20_0_20","openwifi_0_7_1_-33","baseline_error","unnormalised"],style="dotted")
